In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
import datetime as dt
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title: Community Storymap", item_type="Web Mapping Application", max_items=500, outside_org=False)

In [ ]:
import os
today = [dt.date.today()]
log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Communities\Update Demographics"
log_file = os.path.join(log_path, "Update Demographics_" + str(today[0]) + str(".txt"))
out = open(log_file,'w')

for f in find_storymaps:
    print(f.title, f.id)
    try:
        if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            panel_demog = sections[3]
            community = f.title.replace(" Community Storymap", "").replace(' (Unincorporated Community)','')
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            query = "CommunityName = '" + str(community.replace("'","''")) + "'"

        ### DEMOGRAPHICS PANEL ###
            #Historical census table for demographics panel
            print("Updating Demographics...")
            census_base = FeatureLayer("https://maps.commerce.alaska.gov/arcgis/rest/services/Demographics/CDO_Demographics_Census_Population/MapServer/0") #Census locations all locations
            pop_df = census_base.query(where=query).df

            if str(pop_df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                census_tbl = ["<table border='1'><th width='30%'>Census Year</th><th width='45%'>Population</th><tbody>"]
                for y in pop_df.CensusYear:
                    year_query = census_base.query(where="(" + str(query) + ") AND CensusYear = '" + str(y) + "'",out_fields="*")
                    pop = year_query.df.Population[0]
                    census_tbl.append('<tr><td><span style="font-size:14px">' + str(y) + '</span></td><td><span style="font-size:14px">' + str("{:,}".format(pop)) + '</span></td></tr>')
                census_tbl.append('</tbody></table>')
                embed_census_tbl = (' '.join(census_tbl))
                print('Census table updated.')
            else:
                embed_census_tbl = '<i> Demographic data is unavailable for this community. </i>'
                print('Census table unavailable.')
            #Race composition of community as table for demographics panel
            race_base = FeatureLayer("https://maps.commerce.alaska.gov/arcgis/rest/services/Demographics/Alaska_American_Community_Survey/MapServer/1")
            race_data = race_base.query(where="CommunityName LIKE '%" + str(community.replace("'","''")) + "%'").df
            if str(race_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
                embed_poprace = ('<i>Data on race is unavailable for ' + str(community) + '.</i>')
            else:
                race_pop = race_data.TotalPop[0]
                if int(race_pop) != 0:
                    demog_race = ["<table border = '1'><tr><th width='70%'>Race</th><th width='30%'>Percent of Population</th></tr><tbody>",
                                  "<tr><td><span style='font-size:14px'>American Indian or AK Native</span></td><td><span style='font-size:14px'>" + str(round(int(race_data.AmInd_AKNat[0])/race_pop*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>Asian</span></td><td><span style='font-size:14px'>" + str(round(int(race_data.Asian[0])/race_pop*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>Black or African American</span></td><td><span style='font-size:14px'>" + str(round(int(race_data.Black_AfricanAmerican[0])/race_pop*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>Native Hawaiian or Pacific Islander</span></td><td><span style='font-size:14px'>" + str(round(int(race_data.NatHI_PacIsland[0])/race_pop*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>White</span></td><td><span style='font-size:14px'>" + str(round(int(race_data.White[0])/race_pop*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>Other Race</span></td><td><span style='font-size:14px'>" + str(round(int(race_data.OtherRace[0])/race_pop*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>Two or More Races</span></td><td><span style='font-size:14px'>" + str(round(int(race_data.TwoOrMore[0])/race_pop*100,2)) + "%</span></td></tr>"]
                    embed_poprace = (' '.join(demog_race))
                else:
                    embed_poprace = ('<i>Data on race is unavailable for ' + str(community) + '.</i>')

            #Age composition
            age_base = FeatureLayer("https://maps.commerce.alaska.gov/arcgis/rest/services/Demographics/Alaska_American_Community_Survey/MapServer/5")
            age_data = age_base.query(where=query).df
            if str(age_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
                embed_popage = ('<i>Data on age is unavailable for ' + str(community) + '.</i>')
            else:
                total_pop_acs = int(age_data.estimate_totalpop[0])
                if total_pop_acs != 0:
                    demog_age = ["<table border = '1'><tr><th width='70%'>Age</th><th width='30%'>Percent of Population</th></tr><tbody>",
                                  "<tr><td><span style='font-size:14px'>Under 5 years of age</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate_under5__estimate_under[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>5 to 9</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate5_9[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>10 to 14</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate10_14[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>15 to 19</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate15_19[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>20 to 24</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate20_24[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>25 to 34</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate25_34[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>35 to 44</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate35_44[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>45 to 54</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate45_54[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>55 to 59</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate55_59[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>60 to 64</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate60_64[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>65 to 74</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate65_74[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>75 to 84</span></td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate75_84[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                  "<tr><td><span style='font-size:14px'>Over 85 years of age</td><td><span style='font-size:14px'>" + str(round(int(age_data.estimate85plus[0])/total_pop_acs,2)) + "%</span></td></tr>",]
                    embed_popage = (' '.join(demog_age))
                else:
                    embed_popage = ('<i>Data on age is unavailable for ' + str(community) + '.</i>')
            #Sex composition
            m_f_base = FeatureLayer("https://maps.commerce.alaska.gov/arcgis/rest/services/Demographics/Alaska_American_Community_Survey/MapServer/0")
            sex_data = m_f_base.query(where=query).df
            if str(sex_data) != 'Empty DataFrame\nColumns: []\nIndex: []':    
                total_pop_acs = int(age_data.estimate_totalpop[0])
                print("ACS total pop estimate: " + str(total_pop_acs))
                if int(total_pop_acs) == 0 or int(total_pop_acs) < int(sex_data.female_estimate[0]):
                    embed_popsex = ('<i>Data on sex is unavailable for ' + str(community) + '.</i>')
                else:
                    demog_sex = ["<table border = '1'><tr><th width='70%'>Sex</th><th width='30%'>Percent of Population</th></tr><tbody>",
                                 "<tr><td><span style='font-size:14px'>" + 'Male' + "</span></td><td><span style='font-size:14px'>" + str(round(int(sex_data.female_estimate[0])/total_pop_acs*100,2)) + "%</span></td></tr>",
                                 "<tr><td><span style='font-size:14px'>" + 'Female' + "</span></td><td><span style='font-size:14px'>" + str(round(int(sex_data.male_estimate[0])/total_pop_acs*100,2)) + "%</span></td></tr>"]

                    embed_popsex = (' '.join(demog_sex))
            else:
                embed_popsex = '<i>Data on sex is unavailable for ' + str(community) + '.</i>'
            if embed_popsex == '<i>Data on sex is unavailable for ' + str(community) + '.</i>' and embed_popage == '<i>Data on age is unavailable for ' + str(community) + '.</i>'and embed_poprace == '<i>Data on race is unavailable for ' + str(community) + '.</i>' and embed_census_tbl == '<i> Demographic data is unavailable for this community. </i>':
                panel_demog['content'] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783883308" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p><p>No demographic data available for this community.</p>'
            else:
                panel_demog["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783883308" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>Every June the Commissioner of the Department of Commerce Community Economic Development certifies community population figures. If population data is available for this community, those figures are displayed in the map popup to the right.</p>\n\n<p>&nbsp;</p>\n\n<p>Census Population Counts: </p>\n\n<p><div>' + str(embed_census_tbl) + '</div><p>&nbsp;</p><div><p><b> Current Population by Race:</b></p><p>' + str(embed_poprace) +'</p></table></div><p>&nbsp;</p><div><p><b> Current Population by Age:</b></p><p>' + str(embed_popage) + '</table></p></div>\n\n<p>&nbsp;</p><div><p><b> Current Population by Sex*:</b></p><p>' + str(embed_popsex) + '</table></p></div>\n\n<p>&nbsp;</p><p><i>*From the US Census Bureau on sex versus gender: "In general discussions, the concept of gender is often confused with the concept of sex, and the terms are used interchangeably. The meanings of these two concepts are not the same: sex is based on the biological attributes of men and women (chromosomes, anatomy, hormones), while gender is a social construction whereby a society or culture assigns certain tendencies or behaviors to the labels of masculine or feminine." </i></p><p>&nbsp;</p><p>More information about population trends can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=577407acfbc6433389006d099cb25971" target="_blank">here.&nbsp;</a></p>\n'
        
        print('Demographics updated! Yay!')
        csmap.save()
        
    except Exception as e:
        print("----ERROR WITH " + str(community) + " STORYMAP. EXCEPTION: " + str(e) + "----")
        out.write("----ERROR WITH " + str(community) + " STORYMAP. EXCEPTION: " + str(e) + "----")
        out.write("\n\n")
        continue
        
out.close()       
print("Storymap updates complete!")